In [1]:
import os
import sys
import tarfile
import tensorflow as tf
import numpy as np
from scipy import ndimage
from six.moves import cPickle as pickle
from six.moves.urllib.request import urlretrieve
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
top_data_folder = './data/notMNIST/'
dataset_small = 'notMNIST_small'
dataset_large = 'notMNIST_large'

logs_folder = './logs'

if not os.path.exists(top_data_folder):
    os.makedirs(top_data_folder)
    
def path_to(f):
    return os.path.join(top_data_folder, f)
    
if not os.path.exists(logs_folder):
    os.makedirs(logs_folder)

In [3]:
class DownloadProgress:
    def __init__(self):
        self.last_percent_reported = None

    def __call__(self, count, blockSize, totalSize):
        percent = int(count * blockSize * 100 / totalSize)

        if self.last_percent_reported != percent:
            if percent % 5 == 0:
                sys.stdout.write("%s%%" % percent)
                sys.stdout.flush()
            else:
                sys.stdout.write(".")
                sys.stdout.flush()
      
            self.last_percent_reported = percent

In [4]:
def download_and_extract(dataset_name):
    dataset_archive = dataset_name + '.tar.gz'
    filename = path_to(dataset_archive)
    if not os.path.exists(filename):
        url = 'http://commondatastorage.googleapis.com/books1000/' + dataset_archive
        urlretrieve(url, filename, reporthook=DownloadProgress())

    letters_folder = path_to(dataset_name)

    if not os.path.exists(letters_folder):
        with tarfile.open(filename) as tar:
            tar.extractall(path=top_data_folder)

In [5]:
download_and_extract(dataset_small)

In [6]:
download_and_extract(dataset_large)

In [7]:
image_size = 28
class_count = 10
image_channels = 1

def load_folder(folder):
    image_filenames = [f for f in os.listdir(folder) if not os.path.isdir(os.path.join(folder, f))]
    image_count = len(image_filenames)
    
    dataset = np.ndarray(shape=(image_count, image_size, image_size, image_channels), dtype=np.float32)
    
    image_counter = 0
    
    for f in image_filenames:
        image_filename = os.path.join(folder, f)
        try:
            image_data = np.expand_dims(ndimage.imread(image_filename).astype(float), axis=2)
            dataset[image_counter, :, :] = image_data
            image_counter += 1
        except IOError as e:
            # There are plenty of images, I skip this one
            pass
        
    dataset = dataset[0:image_counter, :, :]
    
    print('Full dataset tensor for %s:' % folder, dataset.shape)
    
    return dataset

In [8]:
def get_complete_dataset(dataset_name):
    image_depth = 256
    
    pickle_file = path_to(dataset_name + '_conv.pickle')
    
    if not os.path.exists(pickle_file):
        image_pixels = []
        image_labels = []
        image_labels_one_hot = []
        
        for i, letter in enumerate("ABCDEFGHIJ"):
            letter_dataset = (load_folder(path_to(os.path.join(dataset_name, letter))) * 2 - image_depth) / image_depth
            
            letter_image_count = len(letter_dataset)
            
            letter_labels = np.ones(shape=(letter_image_count,), dtype=np.float32) * i
            letter_labels_one_hot = np.ndarray(shape=(letter_image_count, class_count), dtype=np.float32)
            
            label_one_hot = np.zeros(shape = (class_count,), dtype=np.float32)
            label_one_hot[i] = 1.0
            letter_labels_one_hot[:] = label_one_hot
            
            image_pixels.append(letter_dataset)
            image_labels.append(letter_labels)
            image_labels_one_hot.append(letter_labels_one_hot)
            
        dataset = { 'pixels': np.concatenate(image_pixels), 
                    'labels': np.concatenate(image_labels),
                    'labels_one_hot': np.concatenate(image_labels_one_hot) }
            
        with open(pickle_file, 'wb') as f:
            pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
            
        return dataset
    else:
        with open(pickle_file, 'rb') as f:
            return pickle.load(f)

In [9]:
def split(dataset_name, test_size=0.2, validate_size=0.25, random_state=42):
    dataset = get_complete_dataset(dataset_name)
    answer = {}
    
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    train_validate_indices, test_indices = list(sss1.split(dataset['pixels'], dataset['labels']))[0]

    answer['X_test'] = dataset['pixels'][test_indices]
    answer['y_test'] = dataset['labels_one_hot'][test_indices]

    train_validate_pixels = dataset['pixels'][train_validate_indices]
    train_validate_labels = dataset['labels'][train_validate_indices]
    train_validate_labels_one_hot = dataset['labels_one_hot'][train_validate_indices]
    
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=validate_size, random_state=random_state * random_state)
    train_indices, validate_indices = list(sss2.split(train_validate_pixels, train_validate_labels_one_hot))[0]
    
    answer['X_train'] = train_validate_pixels[train_indices]
    answer['y_train'] = train_validate_labels_one_hot[train_indices]
    
    answer['X_validate'] = train_validate_pixels[validate_indices]
    answer['y_validate'] = train_validate_labels_one_hot[validate_indices]
    
    return answer    

In [10]:
split_dataset = split(dataset_small)
#split_dataset = split(dataset_large)

In [11]:
for k in split_dataset.keys():
    print(k, split_dataset[k].shape)

('X_test', (3745, 28, 28, 1))
('X_train', (11234, 28, 28, 1))
('X_validate', (3745, 28, 28, 1))
('y_validate', (3745, 10))
('y_train', (11234, 10))
('y_test', (3745, 10))


In [12]:
batch_size = 32

patch_size_1 = 7
depth_1 = 16

patch_size_2 = 5
depth_2 = 8

stride_1 = 2
stride_2 = 2

strides_1 = [1, stride_1, stride_1, 1]
strides_2 = [1, stride_2, stride_2, 1]

hidden_node_count= 64

learning_rate = 0.01

In [13]:
graph = tf.Graph()

with graph.as_default():
    # Placeholders for train datasets/labels
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, image_channels),
                                     name='train_dataset')
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, class_count), name='train_labels')
    
    # Constants for validate/test data
    tf_validate_dataset = tf.constant(split_dataset['X_validate'], name='validate_dataset')
    tf_validate_labels = tf.constant(split_dataset['y_validate'], name='validate_labels')
    tf_test_dataset = tf.constant(split_dataset['X_test'], name='test_dataset')
    tf_test_labels = tf.constant(split_dataset['y_test'], name='test_labels')
    
    # Variables
    layer_1_conv_filter = tf.Variable(tf.truncated_normal([patch_size_1, patch_size_1, image_channels, depth_1], stddev=0.1),
                                     name='layer_1_conv_filter')
    layer_1_biases = tf.Variable(tf.zeros([depth_1]), name='layer_1_biases')
    layer_2_conv_filter = tf.Variable(tf.truncated_normal([patch_size_2, patch_size_2, depth_1, depth_2], stddev=0.1), 
                                      name='layer_2_conv_filter')
    layer_2_biases = tf.Variable(tf.ones([depth_2]), name='layer_2_biases')
    
    layer_1_output_size = image_size // stride_1
    layer_2_output_size = layer_1_output_size // stride_2
    
    layer_3_weights = tf.Variable(tf.truncated_normal([layer_2_output_size * layer_2_output_size * depth_2, hidden_node_count],
                                                     stddev=0.1), name='layer_3_weights')
    layer_3_biases = tf.Variable(tf.ones([hidden_node_count]), name='layer_3_biases')
    
    layer_4_weights = tf.Variable(tf.truncated_normal([hidden_node_count, class_count], stddev=0.1), name='layer_4_weights')
    layer_4_biases = tf.Variable(tf.ones(class_count), name='layer_4_biases')
    
    # Model
    def model(data):
        conv1 = tf.nn.conv2d(data, layer_1_conv_filter, strides_1, padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer_1_biases)
        conv2 = tf.nn.conv2d(hidden1, layer_2_conv_filter, strides_2, padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer_2_biases)
        hidden2_shape = hidden2.get_shape().as_list()
        hidden2_reshaped = tf.reshape(hidden2, [hidden2_shape[0], hidden2_shape[1] * hidden2_shape[2] * hidden2_shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(hidden2_reshaped, layer_3_weights) + layer_3_biases)
        output = tf.nn.relu(tf.matmul(hidden3, layer_4_weights) + layer_4_biases)
        return output
       
    # Train
    logits_train = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_train, labels=tf_train_labels))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
    # Predictions
    prediction_train = tf.nn.softmax(logits_train)
    prediction_validate = tf.nn.softmax(model(tf_validate_dataset))
    prediction_test = tf.nn.softmax(model(tf_test_dataset))
    
    def accuracy(predictions, labels):
        correct_predictions = tf.equal(tf.argmax(predictions, 1), tf.argmax(labels, 1))
        return 100.0 * tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    
    # Accuracies
    accuracy_train = accuracy(prediction_train, tf_train_labels)
    accuracy_validate = accuracy(prediction_validate, tf_validate_labels)
    accuracy_test = accuracy(prediction_test, tf_test_labels)
    
    # Summaries
    tf.scalar_summary("Minibatch Loss", loss)
    tf.scalar_summary("Minibatch Accuracy", accuracy_validate)
    tf.scalar_summary("Validate Accuracy", accuracy_validate)
    tf.scalar_summary("Test Accuracy", accuracy_test)
    
    summary_op = tf.merge_all_summaries()

In [14]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    writer = tf.train.SummaryWriter(logs_folder, graph=graph)
    
    session.run(tf.initialize_all_variables())
    print("Initialized")
    
    for step in range(num_steps):
        offset = (step * batch_size) % (split_dataset['y_train'].shape[0] - batch_size)
    
        batch_data = split_dataset['X_train'][offset:(offset + batch_size), :]
        batch_labels = split_dataset['y_train'][offset:(offset + batch_size), :]
    
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, summary, acc_tr, acc_val, acc_te = session.run(
            [optimizer, loss, summary_op, 
             accuracy_train, accuracy_validate, accuracy_test], 
            feed_dict=feed_dict)
        
        writer.add_summary(summary, step)
        
        if (step % 100 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.2f%%" % acc_tr)
            print("Validation accuracy: %.2f%%" % acc_val)
    print("Test accuracy: %.2f%%" % acc_te)

Initialized
Minibatch loss at step 0: 3.068870
Minibatch accuracy: 15.62%
Validation accuracy: 11.03%
Minibatch loss at step 100: 1.210921
Minibatch accuracy: 68.75%
Validation accuracy: 64.54%
Minibatch loss at step 200: 0.429501
Minibatch accuracy: 84.38%
Validation accuracy: 77.12%
Minibatch loss at step 300: 0.448068
Minibatch accuracy: 87.50%
Validation accuracy: 81.42%
Minibatch loss at step 400: 0.874682
Minibatch accuracy: 78.12%
Validation accuracy: 77.41%
Minibatch loss at step 500: 0.626475
Minibatch accuracy: 78.12%
Validation accuracy: 85.37%
Minibatch loss at step 600: 0.470368
Minibatch accuracy: 81.25%
Validation accuracy: 85.87%
Minibatch loss at step 700: 0.420973
Minibatch accuracy: 90.62%
Validation accuracy: 86.09%
Minibatch loss at step 800: 0.468707
Minibatch accuracy: 84.38%
Validation accuracy: 85.47%
Minibatch loss at step 900: 0.218820
Minibatch accuracy: 93.75%
Validation accuracy: 85.07%
Minibatch loss at step 1000: 0.387154
Minibatch accuracy: 93.75%
Valid